In [9]:
# -*- coding: utf-8 -*-
import pyodbc
import pandas as pd
import numpy as np
import os.path
import uuid
import re
#from datetime import time
%matplotlib inline



In [48]:
um = pd.read_excel('52nd Heckerling Institute USB.xlsx', sheetname='Sessions', header=0,)
speak = pd.read_excel('52nd Heckerling Institute USB.xlsx', sheetname='Speakers', header=0 )
#print um.dtypes
speak = speak.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
speak['bio'] = speak.SpeakerBio.str.replace('.pdf', '.html')
um = um.fillna('')
speak = speak.fillna('')
um['speakers'] = um[um.columns[7:13]].applymap(lambda x: [x] if (x != '') else []).sum(1).tolist()
uid = []
for row in um['ProgramTitle']:
    uid.append(uuid.uuid4())
um['UID'] = uid
um['UID'] = um['UID'].astype(str)
cols = um[um.columns[7:13]]
um.drop(cols, axis=1, inplace = True)
#um = um.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [43]:
# read in biographies from html.
bx = []
for sp in speak.bio:
    if os.path.isfile(sp):
        f = open(str(sp),'r')
        message = f.read()
        #print(message)
        f.close()
        bx.append(message)
    else:
        bx.append('')
speak['biotext'] = bx  
speak['biotext'] = speak.biotext.str.replace(r"\n  <title.+/title>", "")

# Concatenate speaker full name to "speaker" column for matching purposes.
cat = []
for row in speak.itertuples():
    if row.SpeakerMiddle:
        cat.append(row.SpeakerFirstName + ' ' + row.SpeakerMiddle + ' ' + row.SpeakerLastName)
    else:
        cat.append(row.SpeakerFirstName  + ' ' + row.SpeakerLastName)
        
speak['speaker'] = cat


In [49]:
um.to_json('umtest.json',orient='records',date_format='iso' )

In [45]:
speak.to_json('speakers.json',orient='records' )

In [44]:
speak.biotext[20]

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">\n<html>\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <meta http-equiv="Content-Style-Type" content="text/css">\n  <meta name="Author" content="Huffman, Jennifer\x00\x00">\n  <meta name="LastAuthor" content="Huffman, Jennifer\x00\x00">\n  <meta name="Company" content="Microsoft\x00\x00">\n  <meta name="Subject" content=" \x00\x00">\n  <meta name="Keywords" content=" \x00\x00">\n  <meta name="CreationTime" content="2017-07-26T10:53:00Z">\n  <meta name="ModificationTime" content="2017-07-26T10:53:00Z">\n  <meta name="Generator" content="Cocoa HTML Writer">\n  <meta name="CocoaVersion" content="1348.17">\n  <style type="text/css">\n    p.p1 {margin: 0.0px 0.0px 0.0px 0.0px; text-align: center; font: 12.0px \'Times New Roman\'; color: #000000}\n    p.p2 {margin: 0.0px 0.0px 0.0px 0.0px; text-align: center; line-height: 23.0px; font: 12.0px \'Times New Roman\'}\n  